In [2]:
from StableMath import StableMath
from util import *

In [4]:
def calcBptInGivenExactTokensOut(amplificationParameter: int, balances: list, amountsOut: list, bptTotalSupply: int, swapFee: int) -> int:
    """
    >>> t = StableMath()
    >>> t.calcBptInGivenExactTokensOut(2,[222,3112,311],[11,22,310],2,4) == 2
    True
    """

    # get current invariant
    currentInvariants = StableMath.calculateInvariant(amplificationParameter, balances)

    # calculate the sum of all token balances
    sumBalances = Decimal(0)
    for i in range(len(balances)):
        sumBalances += balances[i]

    # calculating the weighted balance ratio without considering fees
    tokenBalanceRatiosWithoutFee = [None] * len(balances)
    weightedBalanceRatio = Decimal(0)

    getcontext().prec = 18
    for i in range(len(balances)):
        currentWeight = divUp(balances[i], sumBalances)
        tokenBalanceRatiosWithoutFee[i] = balances[i] - divUp(amountsOut[i], balances[i])
        weightedBalanceRatio = weightedBalanceRatio + mulUp(tokenBalanceRatiosWithoutFee[i], currentWeight)

    # second loop calculates new amounts in, taking into account
    # the fee on the percentage excess
    newBalances = []
    for i in range(len(balances)):
        tokenBalancePercentageExcess = 0
        if weightedBalanceRatio <= tokenBalanceRatiosWithoutFee[i]:
            tokenBalancePercentageExcess = 0
        else:
            tokenBalancePercentageExcess = weightedBalanceRatio - Decimal(divUp(tokenBalanceRatiosWithoutFee[i], complement(tokenBalanceRatiosWithoutFee[i])))

        swapFeeExcess = mulUp(swapFee, tokenBalancePercentageExcess)
        amountOutBeforeFee = Decimal(divUp(amountsOut[i], complement(swapFeeExcess)))
        newBalances[i] = balances[i] - amountOutBeforeFee

    # get the new invariant, taking into account swap fees
    newInvariant = calculateInvariant(amplificationParameter, newBalances)

    # return amountBPTIn
    return bptTotalSupply * Decimal(divUp(newInvariant, complement(currentInvariants)))


In [5]:
StableMath.calcBptInGivenExactTokensOut(
    Decimal(2),
    [Decimal(222),Decimal(3112),Decimal(311)],
    [Decimal(11),Decimal(22),Decimal(310)],
    Decimal(2),Decimal(4)
    ) #is supposed to be 2 

0